# Add Embeddings to an Existing Table


<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/tutorials/add-embeddings.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/add-embeddings.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

In this example we will extend an existing table with embeddings computed from a pre-trained model.

- Write an initial table containing a single column of image URLs.
- Write a new table containing the input URLs and the embeddings computed from a pre-trained model.
- Apply dimensionality reduction to the extended table to get a final table containing the URLs, the embeddings, and the reduced embeddings.

In [ ]:
from transformers import ViTImageProcessor, ViTModel
import torch
from torchvision import transforms
from pathlib import Path
import tlc
import tqdm

## Write the initial table

We write a simple table containing a single column of image URLs from our COCO-128 dataset.

In [ ]:
data_path = Path("../data/coco128/images").absolute().as_posix()
project_name = "add-embeddings"
dataset_name = "coco128"

table = tlc.Table.from_image_folder(
    data_path,
    include_label_column=False,
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
    add_weight_column=False,
    description="COCO128 dataset",
)

image_paths = [row["image"] for row in table.table_rows]

## Extend the table with embeddings from a pre-trained model

We will use the ViT model pre-trained on ImageNet to compute embeddings for the images in the table.
A benefit of using this model is that meaningful embeddings can be extracted easily using the `last_hidden_state` attribute of the model output.

In [ ]:
# Load the model and feature extractor

model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTModel.from_pretrained(model_name).to(device)

# The warning about 'vit.pooler.dense.bias' and 'vit.pooler.dense.weight' being newly initialized
# is not problematic for this use case because we are only extracting embeddings from the last
# hidden state and do not rely on the pooler layer.

In [ ]:
# The input table returns rows of the form {"image": "image_path"}
# Define a map function on the table that returns the images as plain tensors instead

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std),
])

table.map(lambda row: preprocess(row.convert("RGB")))

In [ ]:
# Define a TableWriter to write the embeddings-table

extended_table_writer = tlc.TableWriter(
    table_name="added-embeddings",
    dataset_name=dataset_name,
    project_name=project_name,
    description="COCO128 dataset with added embeddings",
    column_schemas={
        "image": tlc.ImagePath,                                                  # Path to the image (copied from input table)
        "embedding": tlc.Schema(
            value=tlc.Float32Value(number_role=tlc.NUMBER_ROLE_NN_EMBEDDING),    # We assign a special role to the embedding column so that it will be automatically selected for dimensionality reduction
            size0=tlc.DimensionNumericValue(768, 768),                           # The embedding size is 768
            sample_type="hidden",                                                # We don't want the embedding to be displayed in the "sample-view" of the table
            writable=False,                                                      # We do not allow editing the embedding values after they have been computed
        ),
    },
)

In [ ]:
# Create a DataLoader to iterate over the images in batches for faster inference

batch_size = 4

dataloader = torch.utils.data.DataLoader(
    table,
    batch_size=batch_size,
    num_workers=0,
    shuffle=False,  # We don't shuffle the images in order to associate the tensor images with the image paths
)

batched_filenames = (
    image_paths[i * batch_size : (i + 1) * batch_size] for i in range(len(dataloader))
)

In [ ]:
# Run inference on the images and write the embeddings to the extended table's TableWriter
for image_batch, image_path_batch in tqdm.tqdm(
    zip(dataloader, batched_filenames),
    total=len(dataloader),
    desc="Running inference on batches",
):
    with torch.no_grad():
        outputs = model(image_batch.to(device))
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().squeeze().numpy()

    extended_table_writer.add_batch(
        {
            "image": image_path_batch,
            "embedding": embeddings.tolist(),
        }
    )

extended_table = extended_table_writer.finalize()

print(
    extended_table[0].keys()
)  # Notice the "embeddings" column is not present in the sample-view of the table
print(
    extended_table.table_rows[0].keys()
)  # Notice the "embeddings" column is present in the "row-view" of the table

## Reduce the embeddings to 2 dimensions

Finally we reduce the embedding-column to 2 dimensions using UMAP. The result is a table containing the URLs, the embeddings, and the reduced embeddings.

In [ ]:
reduced_table = tlc.reduce_embeddings(
    extended_table,
    method="umap",
    n_components=2,
    metric="euclidean",
    retain_source_embedding_column=True,
)

print(reduced_table.table_rows[0].keys()) # The row-view of the reduced table contains both the embeddings and the reduced embeddings